In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#url 내에서 상세 정보 크롤링 해보기
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
browser = webdriver.Chrome(options=options)
browser.maximize_window()

txt = pd.read_csv('LCK_2019_Summer_Playoffs_match_links.txt', header=None)
url_list = [i for i in txt.iloc[:,0]]
url = url_list[0]

browser.get(url)

In [3]:
#사이드창 닫기
left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
if left_close_button.get_attribute('innerText') == '닫기':
    left_close_button.click()
    
right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
if right_close_button.get_attribute('innerText') == '닫기':
    right_close_button.click()

In [4]:
#text를 통해 얻을 수 있는 정보들(level, player_name, mvp, kda, damage, dpm, dtpm, sight, cs)
text_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\n')
text_data = [i.strip().split('\n\t\n') for i in text_data[1:len(text_data)//2+1]] + [i.strip().split('\n\t\n') for i in text_data[len(text_data)//2+1:]]

data = []
for i in range(len(text_data)):
        level = text_data[i][0]
        player_name = text_data[i][1].split('\n')
        try:
                if player_name[1]:
                        mvp = True
        except:
                mvp = False
        player_name = player_name[0]
        k,d,a = text_data[i][2].split('\n')[0].split(' / ')
        kda_score = text_data[i][2].split('\n')[1]
        damage = text_data[i][3]
        dpm = text_data[i][4]
        dtpm = text_data[i][5]
        sight = text_data[i][6].split(sep='\n')[0]
        ward_install = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[0]
        ward_erase = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[1]
        cs = text_data[i][7].split(sep='\n')[0]
        cs_per_minute = text_data[i][7].split(sep='\n')[1]
        
        data.append([player_name, mvp, level, k, d, a, kda_score, damage, dpm, dtpm, 
                     sight, ward_install, ward_erase, cs, cs_per_minute])
        
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,SKT Khan,False,18,2,0,4,7.19,"8,312",244,346,7,18,5,337,9.9
1,SKT Clid,True,16,4,0,5,10.8,"11,652",342,428,10,45,26,211,6.2
2,SKT Faker,False,18,3,0,2,6,"10,382",305,543,7,19,17,361,10.6
3,SKT Teddy,False,17,1,0,4,6,"25,873",760,266,7,19,13,417,12.3
4,SKT Effort,False,13,0,0,5,6,"1,961",58,148,20,77,19,19,0.6
5,GRF Doran,False,16,0,4,0,0,"8,309",244,625,7,17,4,296,8.7
6,GRF Tarzan,False,14,0,1,0,0,886,26,538,24,28,34,178,5.2
7,GRF Chovy,False,17,0,3,0,0,"7,173",211,382,5,16,11,335,9.8
8,GRF Viper,False,17,0,1,0,0,"8,761",257,375,6,14,15,454,13.3
9,GRF Lehends,False,12,0,1,0,0,"2,174",64,486,20,63,10,63,1.9


In [5]:
#tag를 통해 얻을 수 있는 정보(champion, summoner_skill, rune, items)
data = []
tag_data = browser.find_element(By.XPATH, '//*[@class="py-4"]/div/ul').find_elements(By.TAG_NAME, 'tbody')
tag_data_list = [i.find_elements(By.TAG_NAME, 'tr') for i in tag_data]
tag_data = tag_data_list[0] + tag_data_list[1]   #선수별 데이터 10개
for tag in tag_data:
    champion_data = tag.find_elements(By.TAG_NAME, 'td')[0]
    champion = str(champion_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
    summoner_skill = sorted([tag.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), 
            str(tag.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))]) 
    rune = tag.find_element(By.XPATH, './td[1]/div/div[1]/div[2]').find_elements(By.TAG_NAME, 'img')
    rune = [i.get_attribute('alt') for i in rune]
    
    items = tag.find_elements(By.TAG_NAME, 'td')[-1]
    items = items.find_elements(By.TAG_NAME, 'img')
    items = [item.get_attribute('alt') for item in items]
    
    data.append([champion, summoner_skill, rune, items])
    
pd.DataFrame(data)

,0,1,2,3
0,Akali,"[순간이동, 점멸]","[기민한 발놀림, 결의]","[라바돈의 죽음모자, 방출의 마법봉, 헤르메스의 발걸음, 제어 와드, 존야의 모래시..."
1,Elise,"[강타, 점멸]","[감전, 정밀]","[초시계, 제어 와드, 마법사의 신발, 마법공학 로켓 벨트, 밴시의 장막, 예언자의..."
2,Renekton,"[순간이동, 점멸]","[집중 공격, 영감]","[초시계, 망령의 두건, 헤르메스의 발걸음, 스테락의 도전, 칠흑의 양날 도끼, 투..."
3,Ezreal,"[순간이동, 점멸]",[마법],"[명석함의 아이오니아 장화, 삼위일체, 헤르메스의 시미터, 몰락한 왕의 검, 무라마..."
4,Rakan,"[점멸, 회복]","[수호자, 정밀]","[천 갑옷, 기동력의 장화, 수은 장식띠, 점화석, 예언자의 렌즈]"
5,Mordekaiser,"[순간이동, 점멸]","[정복자, 지배]","[도란의 반지, 제어 와드, 마법사의 신발, 마법공학 로켓 벨트, 라일라이의 수정홀..."
6,Sejuani,"[강타, 점멸]","[여진, 정밀]","[루비 수정, 헤르메스의 발걸음, 제어 와드, 점화석, 가고일 돌갑옷, 예언자의 렌즈]"
7,Sylas,"[순간이동, 점멸]","[착취의 손아귀, 마법]","[라바돈의 죽음모자, 방출의 마법봉, 헤르메스의 발걸음, 도란의 반지, 마법공학 로..."
8,Xayah,"[순간이동, 점멸]","[치명적 속도, 마법]","[주문포식자, 판금 장화, 수호 천사, 무한의 대검, 고속 연사포, 정수 약탈자, ..."
9,TahmKench,"[점멸, 회복]","[수호자, 마법]","[장화, 제어 와드, 수은 장식띠, 지크의 융합, 예언자의 렌즈]"


In [6]:
#받은 데미지
xpath = "//span[contains(text(), '받은 데미지')]"
damage_button = browser.find_element(By.XPATH, xpath)
damage_button.click()

damage_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'li')
damage_received = [i.get_attribute('innerText').split('\n')[-1] for i in damage_data]
damage_received

['11.8k',
 '14.6k',
 '18.5k',
 '9.1k',
 '5.0k',
 '21.3k',
 '18.3k',
 '13.0k',
 '12.8k',
 '16.5k']

In [7]:
browser.quit()

In [8]:
#현재 디렉토리 내 txt 파일 모두 불러오기(리그, 연도별)
def get_txt_files(path, league, year):
    txt_list = [file for file in os.listdir(path) if (file.endswith('txt') and file.startswith(f'{league}_{year}'))]
    return txt_list

def get_urls(txt):
    try:
        url_list = [i for i in pd.read_csv(txt, header=None).iloc[:,0]]
        return url_list
    except:
        return [None]

In [9]:
#txt 파일 내 모든 url 크롤링하기
def crawling_player_data(txt_list): 
    from tqdm import tqdm
    final_data = []
    failed_url = []
    total_time = 0
    for txt in tqdm(txt_list):
        start_time = time.time()   #소요시간 체크
        
        urls = pd.read_csv(txt, header=None)
        url_list = [i for i in urls.iloc[:,0]]
                
        for url_idx, url in enumerate(tqdm(url_list, desc=txt)):   
            try:
                options = webdriver.ChromeOptions()
                #options.headless = True   
                options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
                browser = webdriver.Chrome(options=options)
                browser.maximize_window()

                browser.get(url)
                time.sleep(1)

                #사이드창 닫기
                left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
                if left_close_button.get_attribute('innerText') == '닫기':
                    left_close_button.click()
                
                right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
                if right_close_button.get_attribute('innerText') == '닫기':
                    right_close_button.click()

                #대회 정보
                tournament_name = txt[:-16]
                
                #경기일
                date = url[-10:]    

                #패치 버전
                xpath = "//div[contains(text(), 'Ver.')]"
                version = browser.find_element(By.XPATH, xpath).get_attribute('innerText')[5:]
                
                #매치 스코어
                time.sleep(1)  #왜자꾸 에러나냐..?
                match_score = browser.find_element(By.XPATH, '//*[@class="font-axiforma relative"]/div[2]').get_attribute('innerText').split('\n')
                match_score = match_score[0] + '(' + match_score[1] + ')' + ' : ' + match_score[3] + '(' + match_score[2] + ')'
                                
                #세트 선택
                set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  
                for set_num in range(len(set_list)):
                    current_set = set_list[set_num]
                    current_set.click()
                    
                    #사이드창 닫기
                    left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
                    if left_close_button.get_attribute('innerText') == '닫기':
                        left_close_button.click()
                        
                    right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
                    if right_close_button.get_attribute('innerText') == '닫기':
                        right_close_button.click()
                    
                    time.sleep(1)    
                    set_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]').get_attribute('innerText').split('\n')
                    
                    #팀명
                    team1_name = set_data[1]
                    team2_name = set_data[4]
                    
                    #경기 시간
                    set_time = set_data[0]

                    #선수별 데이터 크롤링
                    text_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\n')
                    text_data = [i.strip().split('\n\t\n') for i in text_data[1:len(text_data)//2+1]] + [i.strip().split('\n\t\n') for i in text_data[len(text_data)//2+1:]]

                    tag_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'tbody')
                    tag_data_list = [i.find_elements(By.TAG_NAME, 'tr') for i in tag_data]
                    tag_data = tag_data_list[0] + tag_data_list[1]

                    for i, tag in enumerate(tag_data):
                        level = text_data[i][0]
                        player_name = text_data[i][1].split('\n')
                        try:
                                if player_name[1]:
                                        mvp = True
                        except:
                                mvp = False
                        player_name = player_name[0]
                        k,d,a = text_data[i][2].split('\n')[0].split(' / ')
                        kda_score = text_data[i][2].split('\n')[1]
                        damage_inflicted = text_data[i][3]
                        dpm = text_data[i][4]
                        dtpm = text_data[i][5]
                        sight = text_data[i][6].split(sep='\n')[0]
                        ward_install = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[0]
                        ward_erase = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[1]
                        cs = text_data[i][7].split(sep='\n')[0]
                        cs_per_minute = text_data[i][7].split(sep='\n')[1]
                        
                        champion_data = tag.find_elements(By.TAG_NAME, 'td')[0]
                        champion = str(champion_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
                        summoner_skill = sorted([tag.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), 
                                str(tag.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))]) 
                        rune = tag.find_element(By.XPATH, './td[1]/div/div[1]/div[2]').find_elements(By.TAG_NAME, 'img')
                        rune = [i.get_attribute('alt') for i in rune]
                        
                        items = tag.find_elements(By.TAG_NAME, 'td')[-1]
                        items = items.find_elements(By.TAG_NAME, 'img')
                        items = [item.get_attribute('alt') for item in items]
                        
                    xpath = "//span[contains(text(), '받은 데미지')]"
                    damage_button = browser.find_element(By.XPATH, xpath)
                    damage_button.click()

                    damage_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'li')
                    damage_received = [i.get_attribute('innerText').split('\n')[-1] for i in damage_data]

                    final_data.append([tournament_name, date, version, match_score, set_num+1, set_time,  
                                       player_name, mvp, champion, level, summoner_skill, rune,
                                        k, d, a, kda_score, damage_inflicted, damage_received[i], 
                                        dpm, dtpm, sight, ward_install, ward_erase,
                                        cs, cs_per_minute, items])
        
            except:
                failed_url.append(url)
                continue
            browser.close()                    
        end_time = time.time()
        crawling_time = int(end_time - start_time)
        total_time += crawling_time   
        
    if len(failed_url) == 0:
        print('전체 데이터 크롤링 성공!')
    else:
        print(f'url {len(failed_url)}개 크롤링 실패')            
    print(f'총 소요 시간 : {round(total_time / 60, 2)}분')    
    print('-'*100)  

    columns = ['tournament name', 'date', 'version', 'match score', 'set', 'time',  
               'player_name', 'MVP', 'champion', 'level', 'summoner skill', 'rune',
               'K', 'D', 'A', 'KDA score', 'damage_inflicted', 'damage received', 
               'dpm', 'dtpm', 'sight', 'ward install', 'ward_erase',
               'cs', 'cs per minute', 'items']
    index = ['tournament name', 'date', 'version', 'match score', 'set', 'time']
    
    df = pd.DataFrame(final_data, columns=columns).set_index(index) 
        
    return df, failed_url

In [10]:
#함수 테스트
txt_list = get_txt_files('./', 'LCK', '2015')   
print(f'LCK txt 파일 개수 : {len(txt_list)}')

txt_list

LCK txt 파일 개수 : 3


['LCK_2015_Summer_Playoffs_match_links.txt',
 'LCK_2015_Summer_Promotion_match_links.txt',
 'LCK_2015_Summer_Regular season_match_links.txt']

In [11]:
df, failed_url = crawling_player_data(txt_list[:1])  #우선 txt파일 하나만 사용해보기

print(df.shape)
df

100%|██████████| 1/1 [01:12<00:00, 72.71s/it]


url 1개 크롤링 실패
총 소요 시간 : 1.2분
----------------------------------------------------------------------------------------------------
(9, 20)


player_name  \
tournament name          date       version match score     set time                 
LCK_2015_Summer_Playoffs 2015-08-29 5.14    SKT(3) : KT(0)  1   31:33  KT Piccaboo   
                                                            2   30:28     SKT Wolf   
                                                            3   40:53  KT Piccaboo   
                         2015-08-19 5.14    CJE(0) : KOO(3) 1   49:42  KOO GorillA   
                                                            2   26:22  CJE MadLife   
                                                            3   43:10  KOO GorillA   
                         2015-08-16 5.14    KOO(2) : KM(1)  1   32:12      KM Pure   
                                                            2   34:19  KOO GorillA   
                                                            3   33:11      KM Pure   

                                                                         MVP  \
tournament name          date       version match score     set time           
LCK_2015_Summer_Playoffs 2015-08-29 5.14    SKT(3) : KT(0)  1   31:33  False   
                                                            2   30:28  False   
                                                            3   40:53  False   
                         2015-08-19 5.14    CJE(0) : KOO(3) 1   49:42  False   
                                                            2   26:22  False   
                                                            3   43:10  False   
                         2015-08-16 5.14    KOO(2) : KM(1)  1   32:12  False   
                                                            2   34:19  False   
                                                            3   33:11  False   

                                                                      champion  \
tournament name          date       version match score     set time             
LCK_2015_Summer_Playoffs 2015-08-29 5.14    SKT(3) : KT(0)  1   31:33    Braum   
                                                            2   30:28   Kennen   
                                                            3   40:53   Thresh   
                         2015-08-19 5.14    CJE(0) : KOO(3) 1   49:42    Janna   
                                                            2   26:22    Leona   
                                                            3   43:10  Morgana   
                         2015-08-16 5.14    KOO(2) : KM(1)  1   32:12  Alistar   
                                                            2   34:19    Janna   
                                                            3   33:11    Leona   

                                                                      level  \
tournament name          date       version match score     set time          
LCK_2015_Summer_Playoffs 2015-08-29 5.14    SKT(3) : KT(0)  1   31:33    12   
                                                            2   30:28    13   
                                                            3   40:53    14   
                         2015-08-19 5.14    CJE(0) : KOO(3) 1   49:42    18   
                                                            2   26:22    10   
                                                            3   43:10    17   
                         2015-08-16 5.14    KOO(2) : KM(1)  1   32:12    13   
                                                            2   34:19    15   
                                                            3   33:11    12   

                                                                      summoner skill  \
tournament name          date       version match score     set time                   
LCK_2015_Summer_Playoffs 2015-08-29 5.14    SKT(3) : KT(0)  1   31:33       [점멸, 점화]   
                                                            2   30:28       [점멸, 점화]   
                                                            3   40:53       [점멸, 탈진]   
                         2015-

## 리그/연도별 전체 데이터 크롤링

In [14]:
#리그/연도별 크롤링 후 csv 파일로 만들기
leagues = ['LPL', 'LCS', 'LEC']  
years = [str(year) for year in range(2015, 2023)]
failed_urls = []
for league in leagues:
    for year in years:
        txt_list = get_txt_files('./', league, year)
        print(f'{league}_{year} txt 파일 개수 : {len(txt_list)}')

        df, failed_url = crawling_player_data(txt_list)
        df.to_csv(f'{league}_{year}_player_data.csv')
        failed_urls.append(failed_url)

LCK_2022 txt 파일 개수 : 3


100%|██████████| 3/3 [1:07:42<00:00, 1354.26s/it]


전체 데이터 크롤링 성공!
총 소요 시간 : 67.68분
----------------------------------------------------------------------------------------------------


In [13]:
print(len(failed_urls))
failed_urls

2


[['https://qwer.gg/ko/matches/17049/kt-vs-dk-2020-03-25',
  'https://qwer.gg/ko/matches/17160/t1-vs-hle-2020-07-25',
  'https://qwer.gg/ko/matches/17161/af-vs-sp-2020-07-24'],
 []]

### 실패한 URL 다시 크롤링 해보기

In [27]:
final_data = []

txt_list = get_txt_files('./', 'LCK', '2020')
for txt in txt_list:
    urls = pd.read_csv(txt, header=None)
    url_list = [i for i in urls.iloc[:,0]]
        
    for i in failed_urls:
        for url in url_list:
            if i == url:
                options = webdriver.ChromeOptions()
                #options.headless = True   
                options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
                browser = webdriver.Chrome(options=options)
                browser.maximize_window()

                browser.get(url)
                time.sleep(1)

                #사이드창 닫기
                left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
                if left_close_button.get_attribute('innerText') == '닫기':
                    left_close_button.click()
                
                right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
                if right_close_button.get_attribute('innerText') == '닫기':
                    right_close_button.click()

                #대회 정보
                tournament_name = txt[:-16]
                
                #경기일
                date = url[-10:]    

                #패치 버전
                xpath = "//div[contains(text(), 'Ver.')]"
                version = browser.find_element(By.XPATH, xpath).get_attribute('innerText')[5:]
                
                #매치 스코어
                time.sleep(1)  
                match_score = browser.find_element(By.XPATH, '//*[@class="font-axiforma relative"]/div[2]').get_attribute('innerText').split('\n')
                match_score = match_score[0] + '(' + match_score[1] + ')' + ' : ' + match_score[3] + '(' + match_score[2] + ')'
                                
                #세트 선택
                set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  
                for set_num in range(len(set_list)):
                    current_set = set_list[set_num]
                    current_set.click()
                    
                    #사이드창 닫기
                    left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
                    if left_close_button.get_attribute('innerText') == '닫기':
                        left_close_button.click()
                        
                    right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
                    if right_close_button.get_attribute('innerText') == '닫기':
                        right_close_button.click()
                    
                    time.sleep(1)    
                    set_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]').get_attribute('innerText').split('\n')
                    
                    #팀명
                    team1_name = set_data[1]
                    team2_name = set_data[4]
                    
                    #경기 시간
                    set_time = set_data[0]

                    #선수별 데이터 크롤링
                    text_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\n')
                    text_data = [i.strip().split('\n\t\n') for i in text_data[1:len(text_data)//2+1]] + [i.strip().split('\n\t\n') for i in text_data[len(text_data)//2+1:]]

                    tag_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'tbody')
                    tag_data_list = [i.find_elements(By.TAG_NAME, 'tr') for i in tag_data]
                    tag_data = tag_data_list[0] + tag_data_list[1]

                    for i, tag in enumerate(tag_data):
                        level = text_data[i][0]
                        player_name = text_data[i][1].split('\n')
                        try:
                                if player_name[1]:
                                        mvp = True
                        except:
                                mvp = False
                        player_name = player_name[0]
                        k,d,a = text_data[i][2].split('\n')[0].split(' / ')
                        kda_score = text_data[i][2].split('\n')[1]
                        damage_inflicted = text_data[i][3]
                        dpm = text_data[i][4]
                        dtpm = text_data[i][5]
                        sight = text_data[i][6].split(sep='\n')[0]
                        ward_install = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[0]
                        ward_erase = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[1]
                        cs = text_data[i][7].split(sep='\n')[0]
                        cs_per_minute = text_data[i][7].split(sep='\n')[1]
                        
                        champion_data = tag.find_elements(By.TAG_NAME, 'td')[0]
                        champion = str(champion_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
                        summoner_skill = sorted([tag.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), 
                                str(tag.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))]) 
                        rune = tag.find_element(By.XPATH, './td[1]/div/div[1]/div[2]').find_elements(By.TAG_NAME, 'img')
                        rune = [i.get_attribute('alt') for i in rune]
                        
                        items = tag.find_elements(By.TAG_NAME, 'td')[-1]
                        items = items.find_elements(By.TAG_NAME, 'img')
                        items = [item.get_attribute('alt') for item in items]
                        
                    xpath = "//span[contains(text(), '받은 데미지')]"
                    damage_button = browser.find_element(By.XPATH, xpath)
                    damage_button.click()

                    damage_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'li')
                    damage_received = [i.get_attribute('innerText').split('\n')[-1] for i in damage_data]

                    final_data.append([tournament_name, date, version, match_score, set_num+1, set_time,  
                                        player_name, mvp, champion, level, summoner_skill, rune,
                                        k, d, a, kda_score, damage_inflicted, damage_received[i], 
                                        dpm, dtpm, sight, ward_install, ward_erase,
                                        cs, cs_per_minute, items])

                browser.close()                    


player_name  \
tournament name                date       version match score    set time                 
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01    T1 Effort   
                                                                 2   25:23  HLE Lehends   
                                                                 3   26:27    T1 Effort   

                                                                              MVP  \
tournament name                date       version match score    set time           
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01  False   
                                                                 2   25:23  False   
                                                                 3   26:27  False   

                                                                            champion  \
tournament name                date       version match score    set time              
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01  Nautilus   
                                                                 2   25:23  Nautilus   
                                                                 3   26:27  Nautilus   

                                                                           level  \
tournament name                date       version match score    set time          
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01    11   
                                                                 2   25:23    10   
                                                                 3   26:27    12   

                                                                           summoner skill  \
tournament name                date       version match score    set time                   
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01       [점멸, 점화]   
                                                                 2   25:23       [점멸, 탈진]   
                                                                 3   26:27       [점멸, 점화]   

                                                                                rune  \
tournament name                date       version match score    set time              
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01  [여진, 영감]   
                                                                 2   25:23  [여진, 영감]   
                                                                 3   26:27  [여진, 영감]   

                                                                            K  \
tournament name                date       version match score    set time       
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01  0   
                                                                 2   25:23  0   
                                                                 3   26:27  2   

                                                                            D  \
tournament name                date       version match score    set time       
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01  1   
                                                                 2   25:23  1   
                                                                 3   26:27  1   

                                                                            A  \
tournament name                date       version match score    set time       
LCK_2020_Summer_Regular season 2020-07-25 10.14   T1(2) : HLE(1) 1   30:01  4   
                                                                 2   25:23  3   
                                                                 3   26:27  9   

                                                                           KDA score  \
tournament name                date       version match score    set time              
LCK_2020_Summer_Regular season 

In [28]:
len(final_data)

7

### csv 파일에 추가 입력하기

In [30]:
import csv
f = open('LCK_2020_player_data.csv', 'w')
writer = csv.writer(f)
for data in final_data:
    writer.writerow(data)
f.close()